In [7]:
import pandas as pd
import os
import re
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [8]:
def parse_logs(filename):
    with open(filename) as f:
        content = f.readlines()        
        
    content = [x.strip() for x in content] 
    results = []
    
    for line in content:    
        try:
            data = {}
            data['passed'] = bool(re.search("passed", line, flags=re.IGNORECASE))
            data['name'] = re.search("isis3_[A-z]+|[A-z]+_app_[A-z]+", line, flags=re.IGNORECASE).group(0)
            data['test_id'] = int(re.search("#[0-9]+", line, flags=re.IGNORECASE).group(0)[1:])
            data['time'] = float(re.search("[0-9]+\.[0-9]+", line, flags=re.IGNORECASE).group(0))
            results.append(data)
        except Exception as e:
            pass
    
    return pd.DataFrame(results)

In [71]:
def parse_make(filename):
    with open(filename) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
    results = []
    # Set initial data and target to none since the re.match may not find anything
    data = {'Target':None}
    data_list = []
    target = "Generation"
    
    for line in content:
        try:
            # If nothing is found, the .group(0) call will cause an error and
            # move to the exception clause
            target = re.match("Scanning dependencies of target [A-z or 0-9 or _]+|[A-z or 0-9 or _]+.so", line,
                              flags=re.IGNORECASE).group(0)[32:]
            
            if data['Target'] != target:
                if data['Target']:
                    results.append(data)
                    
                data_list = []
                data = {'Target':target, 'data_list':data_list, 'built':True}
                
        except Exception as e:
            # Check for a particular line to indicate the switch to parsing installs
            if line == "Install the project...":
                    target = None
                    results.append(data)
                    data_list = []
                    data = {'Target':'Installs', 'data_list':data_list, 'built':True}
            
            # If there is a target, look for the various regexs under the target
            if target:
                std_line = re.search("\[[0-9 or \ ]*%] Building|\[[0-9 or \ ]*%] Linking|" +
                                     "\[[0-9 or \ ]*%] Built|\[[0-9 or \ ]*%] Generating", 
                                     line, flags=re.IGNORECASE)
                # If there is noting in the std_line, there is some weird output and the
                # target failed to build
                if not std_line and data:
                    data['built'] = False
                    data_list.append(line)
            # No target, we are installing. Look for the various regexs for installing
            else:
                std_line = re.search("Install|-- Install|-- Installing|-- Set runtime path of|" +
                                     "-- Up-to-date", 
                                     line, flags=re.IGNORECASE)
                if std_line:
                    data_list.append(line)
                if not std_line:
                    data_list.append(line)
                    data['built'] = False
                    break
            pass
    results.append(data)
    return pd.DataFrame(results)